# Phi-3

## 概要

Phi-3は、以下のシリーズからなる大規模言語モデル:

- phi-3-mini
- phi-3-small
- phi-3-medium
- phi-3.5-mini
- phi-3.5-MoE
- phi-3.5-Vision

phi-3-miniの特徴:

- 3.8B（38億）パラメータで、スマートフォンに搭載できるほど小さいモデル
- 3.3兆トークンで訓練
    - phi-2で使用したデータセットを拡張
        - 高度にフィルタリングしたウェブデータと合成データ
- Mixtral 8x7BやGPT-3.5に匹敵する性能
    - MMLU 69%
    - MT-bench 8.38
- 頑健性・安全性・アラインメントを行った指示チューニング済みモデルも展開

phi-3-miniをパラメータスケーリング:

- phi-3-small
    - 7B
    - MMLU 75%
    - MT-bench 8.7
- phi-3-medium
    - 14B
    - MMLU 78%
    - MT-bench 8.9

多言語・マルチモーダル・長いコンテキストに対応したphi-3.5も展開:

- phi-3.5-mini
- phi-3.5-MoE
    - $16 \times 3.8$ B
        - うちアクティブパラメータは6.6B
    - Llama 3.1・Mixtralよりも言語推論・数学・コーディングタスクで高い性能
    - Gemini-1.5-Flash・GPT-4O-miniに同等の性能
- phi-3.5-Vision
    - 4.2B
    - 複数画像とテキストからなるプロンプトに対応

## 導入

phiモデルは、データセットの品質を改善させることで性能を向上させてきた

- phi-2では、高度なフィルタリングと合成データにより、パラメータ数が25倍のモデルに匹敵する性能を実現
    - パブリックなウェブデータを大規模言語モデルでフィルタリング
    - 大規模言語モデルで生成した合成データ
- phi-3では、phi-2のデータセットをより大規模かつ高度に改善

## 技術仕様

phi-3-miniは、トランスフォーマーデコーダーアーキテクチャを採用:

- コンテキスト長は4K（4096）トークン
    - LongRopeで拡張したphi-3-mini-128Kのコンテキスト長は128K（128000）トークン
- Llama-2と同じアーキテクチャ・トークナイザーを採用
    - Llama-2モデル向けのライブラリに互換性がある
    - 隠れ層次元は3072
    - アテンションヘッド数は32
    - レイヤー数は32
    - 語彙サイズは32064トークン
    - bfloat16を使用し、合計3.3兆トークンで訓練
- 指示チューニングしたモデルも展開
    - `<|user|>\n Question <|end|>\n <|assistant|>`

phi-3-smallは、パラメータスケーリングしたモデル:

- 多言語対応のため、tiktokenトークナイザーを採用
    - 語彙サイズは 100352トークン
    - コンテキスト長は8K（8092）トークン
- 一般的な7Bモデルのアーキテクチャを採用
    - 隠れ層次元は4096
    - アテンションヘッド数は32
    - レイヤー数は32
    - GEGLU活性化関数を使用
    - GQA（Grouped Query Attention）を採用し、4つのクエリが1つのキーを共有
    - Blocksparseアテンションで、訓練と推論の速度を改善
        - 訓練用にFlashAttention参考にしたカーネルをTritonで実装
        - 推論用に2つのカーネルを実装
            - 入力シーケンスを一度に処理するカーネル
            - vLLMの[PagedAttention][2]組み込んだカーネル
        - 通常のアテンションと交互に配置し、検索性能を維持
- [Maximal Update Parametrization（muP）][1]でハイパーパラメータを探索
    - muPは、小規模なモデル（proxy model）でハイパーパラメータを調整し、転移する手法
- 10%の多言語データ

[1]: https://github.com/microsoft/mup
[2]: https://docs.vllm.ai/en/v0.4.1/dev/kernel/paged_attention.html

Blocksparseアテンションは、キー・バリューをブロックに分けて、遠いブロックに対する計算をまばらにする効率化手法:

![](image/fig1.png)

- ブロック8のクエリトークンを処理しているときの、blocksparseアテンションの計算簡略化の様子
- 青色は、距離が近いローカルブロック
- 黄色は、距離が遠いリモートブロック
- 灰色は、計算をスキップするブロック
- アテンションヘッドごとに注目するリモートブロックが異なる
- KVキャッシュを大幅に削減

phi-3.5-MoEは、phi-3-smallとは異なる方法でパラメータスケーリングしたモデル:

- 16個のエキスパートネットワークからTop2ルーティングする手法を採用
    - エキスパートネットワークは、GLUを採用
    - ルーティングモジュールは、トークンごとに2個のエキスパートネットワークを使用する
    - $16\times 3.8$ B（42B）のパラメータのうち、6.6Bのアクティブパラメータ
- ルーティングモジュールは、[SparseMixer][1]のアプローチを採用
- phi-3-medium・phi-3-miniと同じトークナイザーを使用
    - 語彙サイズは32064トークン

[1]: https://github.com/microsoft/SparseMixer

phi-3-miniを4bit量子化することで、メモリ使用量を1.8GBに削減できる:

- A16 Bionicチップを搭載したiPhone 14で、毎秒12秒トークンを実現

## SparseMixer

SparseMixerは、ルーティングネットワークが確率的にエキスパートを選択する離散的な操作を勾配推定する手法

一般的なMoEのルーティングネットワークの誤差逆伝搬:

- ルーティングのための勾配計算を無視（恒等関数に置き換え）
- 効率的なスパース計算と引き換えに、訓練シグナルを犠牲し、学習が不安定

SparseMixer（[v1][1], [v2][2]）により、微分不可能な勾配計算を効率よく近似する

[1]: https://arxiv.org/abs/2310.00811
[2]: https://arxiv.org/abs/2409.12136

### ルーティングネットワークの勾配計算（Top1）

$N$個のエキスパートを次式で表す:

$$
\{f_{i}(x)\}_{i=1}^{N}
$$

$i$番目のエキスパートに対するゲートの値（選ばれる確率）はSoftmax関数で計算できる:

$$
\pi_{i}=softmax(\theta)_{i}=\frac{exp(\theta_{i})}{\sum_{j=1}^{n}exp(\theta_{j})}
$$

このとき$\theta$は入力にルーターネットワークの重み$W_r$を掛けた値:

$$
\theta=W_{r}\cdot x
$$

エキスパートのインデックス $i\in[1,\cdot\cdot\cdot,N]$ のワンホット表現を $I_i$ とし、$D$ をいずれかの$I_i$をとる離散確率変数とする:

$$
I_{i}\in\mathcal{R}^{N\times1},\quad D\in\{I_{1},\cdot\cdot\cdot,I_{N}\}
$$

MoEの訓練中、$D$ はゲートの確率分布 $\pi$ からサンプリングする（実装はGumbel Softmaxで、確率にノイズを加算して最大のインデックスを求める）:

$$
D\sim\pi
$$

MoEの推論中、$D$ は最もゲートの値が高い $I_i$ がサンプリングする:

$$
D\leftarrow arg~max_{I_{i}}\pi_{I_{i}}
$$

MoEレイヤーの最終出力 $y$ は、ゲートの値とエキスパートの順伝搬を掛け合わされた値:

$$
y=\pi_{D}f_{D}(x)
$$

MoEレイヤーの後のニューラルネットワークを微分可能な関数 $g:\mathcal{R}^{n}\rightarrow\mathcal{R}$ とすると、ルーティングネットワークの損失関数は **すべてのエキスパートに $g$ を適用した期待値** で表される:

$$
\mathcal{L}(W_{r})=E_{D\sim softmax(W_{r}x)}[g(\pi_{D}f_{D}(x))]=\sum_{D}\pi_{D}\cdot g(\pi_{D}f_{D}(x)).
$$

簡単のために、ルーティングネットワークの勾配 $\frac{\partial\mathcal{L}(W_{r})}{\partial W_{r}}$ を $\nabla_0 + \nabla_1$ とすると、次のように展開できる:

$$
\frac{\partial\mathcal{L}(W_{r})}{\partial W_{r}} := \nabla_{0}+\nabla_{1}
$$

$$
\nabla_{0}=\sum_{I_{i}}g(\pi_{I_{i}}f_{I_{i}}(x))\frac{\partial\pi_{I_{i}}}{\partial W_{r}}
$$

$$
\nabla_{1}=\sum_{I_{i}}\pi_{I_{i}}\frac{\partial g(\pi_{I_{i}}f_{I_{i}}(x))}{\partial W_{r}}.
$$

$\nabla_1$ は通常の誤差逆伝搬を計算できるが、$\nabla_0$ は「エキスパートを選ぶ」という離散的な操作のため勾配がゼロになり、誤差逆伝搬を計算できない

現状のMoEでは、$\nabla_0$ を無視して $\nabla_1$ のみがモデルの訓練に使用している

SparseMixerは、$\nabla_0$の計算を「ゼロ（ベースライン$g(0)$からの変化」（どのエキスパートも選ばなかった場合との差分）として捉え直し、ODE（常微分方程式）の数値解析で近似する

誤差逆伝搬を、「$t=0$（$g(0)$）から、$t=1$（$g(A)$）までの変化量を求める」という常微分方程式（ODE）の問題として捉え直す

### 1次精度近似（順方向オイラー法, forward Euler method）

オイラー法は、$t=1$での勾配を使用して全体の変化量を計算する手法

エキスパートの出力 $\partial g(\pi_{D}f_{D}(x))$ をそのまま使った勾配で $\nabla_0$ を近似する:

$$
\hat{\nabla}_{SparseMixer-1st} := \frac{\partial g(\pi_{D}f_{D}(x))}{\partial W_{r}}
$$

### ２次精度近似（中点法, mid-point method）

中点法は、$t=0.5$の勾配を使用して全体の変化量を計算する方法（順伝搬の出力を半分にして勾配を計算し、2倍する）

エキスパートの出力を半分にした中間地点の勾配を計算し、それを2倍することで$\nabla_0$を近似する:

$$
\hat{\nabla}_{SparseMixer-2rd} := 2 \cdot \frac{\partial g\left(\frac{\pi_{D}f_{D}(x)}{2}\right)}{\partial W_{r}}
$$

### SparseMixer-v1

2次精度近似だけ使用すると、訓練時の順伝搬（出力を$1/2$にする）と推論時の順伝搬（出力を$1/2$にしない）でギャップが生じる

サンプリングしたエキスパート $D$ が確率最大のものか否かで、1次精度近似と2次精度近似を使い分ける:

$$
\text{SparseMixer} := (1-\delta_{D}) \hat{\nabla}_{SparseMixer-2rd} + \delta_{D} \hat{\nabla}_{SparseMixer-1st}
$$

$$
\delta_{D}=\begin{cases}1, & \text{if } D=\arg\max \pi_{I_{i}}\\ 0, & \text{otherwise}\end{cases}
$$

推論時は「確率が最大以外が選ばれる」状況は起こらないため、訓練時にサンプリングしたエキスパートが確率最大ではない場合は2次精度近似を使用する

### SparseMixer-v2

SparseMixer-v2は、オイラー法（Euler's method）とホインの3次法（Heun's third-order method）という数値解析を用いてルーティングネットワークの勾配計算を近似する手法

ルーターネットワークの勾配を $\nabla_z$ は、式変形により次式で表される:

$$
\nabla z = \sum_{i=0}^{n-1} \left( p_i \cdot \frac{\partial f(p_i \cdot \text{Expert}(x,w_i))}{\partial z} + (f(p_i \cdot \text{Expert}(x,w_i)) - f(0)) \cdot \frac{\partial p_i}{\partial z} \right)
$$

$(f(p_i \cdot \text{Expert}(x,w_i)) - f(0))$ という差分の計算にすべてのエキスパートの順伝搬が必要になるため、オイラー法とホインの3次法で近似する

オイラー法:

$$
f(p_i \cdot \text{Expert}(x,w_i)) - f(0) \approx f'(p_i \cdot \text{Expert}(x,w_i)) \cdot p_i \cdot \text{Expert}(x,w_i)
$$

ホインの3次法:

$$
f(p_i \cdot \text{Expert}(x,w_i)) - f(0) \approx \left( \frac{1}{4} \cdot f'(p_i \cdot \text{Expert}(x,w_i)) + \frac{3}{4} \cdot f' \left( \frac{p_i \cdot \text{Expert}(x,w_i)}{3} \right) \right) \cdot p_i \cdot \text{Expert}(x,w_i)
$$

オイラー法によるルーティングネットワークの勾配計算結果:

$$
\begin{aligned}
\nabla_{1st}z &= \sum_{i=0}^{n-1} \left( p_i \cdot \frac{\partial f(p_i \cdot \text{Expert}(x,w_i))}{\partial z} + f'(p_i \text{Expert}(x,w_i)) \cdot p_i \text{Expert}(x,w_i) \cdot \frac{\partial p_i}{\partial z} \right) \\
&= E_{D \sim \text{MaskedSoftmax}(z)} \left[ 2 \cdot \frac{\partial f(p_D \cdot \text{Expert}(x,w_D))}{\partial z} \right]
\end{aligned}
$$

ホインの3次法によるルーティングネットワークの勾配の近似計算:

$$
\begin{aligned}
\nabla_{3rd}z &= \sum_{i=0}^{n-1} \left( p_i \cdot \frac{\partial f(p_i \cdot \text{Expert}(x,w_i))}{\partial z} + \left( \frac{1}{4} \cdot f'(p_i \cdot \text{Expert}(x,w_i)) + \frac{3}{4} \cdot f' \left( \frac{p_i \cdot \text{Expert}(x,w_i)}{3} \right) \right) \cdot p_i \cdot \text{Expert}(x,w_i) \cdot \frac{\partial p_i}{\partial z} \right) \\
&= E_{D \sim \text{MaskedSoftmax}(z), B \sim \text{Bernoulli}(\frac{5}{8})} \left[ (6-4B) \cdot \frac{\partial f \left( \frac{1+2B}{3} \cdot p_D \cdot \text{Expert}(x,w_D) \right)}{\partial z} \right]
\end{aligned}
$$

SparseMixer-v2は、オイラー法とホイン法を組み合わせた勾配推定（phi-3.5-MoEの実装で使用）:

$$
\hat{\nabla}_{D,\text{SparseMixer-v2}}z = E_{B \sim \text{Bernoulli}(\frac{1}{2})} [f'(\frac{1+2 \cdot \max(B,\delta_D)}{3} \cdot p_D \cdot \text{Expert}(x,w_D)) \frac{\partial p_D \cdot \text{Expert}(x,w_D)}{\partial z}]
$$

- $\delta_D$: 選択されたエキスパートがスコア1位だったかどうか
- $B$: ランダム生を加えるためのベルヌーイ試行
- $f^\prime(\cdot)$: 出力の勾配

## 訓練方法

phi-1の[Textbooks Are All You Need][1]と同様のアプローチ:

- 訓練データは、教科書レベル（educational level）のデータセットを構成
    - 大規模言語モデルを使用してパブリックなデータセットをフィルタリングし、合成
- 事前学習は2段階で実施
    1. 一般的な知識と言語理解を訓練
        - インターネットのパブリックデータ
    2. 論理的推論と専門知識を訓練
        - 前段階のデータを高度にフィルタリングし、合成
        - 事実だけが書かれたデータを削除し、論理的な推論を促すデータを残した

[1]: https://arxiv.org/abs/2306.11644

phi-3-mediumは、phi-3-miniよりもわずかに多いエポック数で訓練:

- アーキテクチャは同じ
    - 14B
    - ヘッド数は40
    - レイヤー数は40
    - 埋め込み次元は5120
- 3.8Bから7Bの性能向上に比べ、7Bから14Bへの性能向上が小さく課題あり

事後学習は、SFTとDPOの2段階で実施:

- SFTでは、多分野の高品質なデータを使用
    - 数学・コーディング・推論・会話・モデルの自己認識・安全性に関するデータ
    - 英語のみのデータ
- DPOでは、振る舞いを訓練するためのデータを使用
    - チャットフォーマット・推論・責任あるAI（Responsible AI）に関するデータ

## ベンチマーク

![](image/table1.png)

- 総合的なベンチマーク: Phi-3ファミリー（特にMedium）が優勢
    - MMLU (Massive Multitask Language Understanding): 高校から大学までの幅広い知識を問う
    - AGIEval (A Human-Centric Benchmark for Evaluating Foundation Models): SATなど人間向けの標準テスト
    - BigBench-Hard (BBH): モデルが苦手とする困難なタスク群
    - TriviaQA: 雑学クイズ（この項目のみGPT-3.5とMixtralがPhi-3を上回る）
- 常識・推論ベンチマーク: Phi-3ファミリー（特にMedium）が優勢
    - HellaSwag: 日常的な状況の「次」として最も自然な文を選ぶ
    - Arc-C / Arc-E: 小学校レベルの科学的な推論問題（CがChallenge、EがEasy）
    - WinoGrande: 文脈から代名詞が指すものを当てる常識問題
    - PIQA (Physical Interaction QA): 物理的な世界の常識（物の動きなど）を問う
    - SociQA (Social Interaction QA): 社会的な状況での人の意図などを問う
    - OpenBookQA: 少量の事実（教科書）を元に推論する科学問題
    - BoolQ: 短文を読み「はい/いいえ」で答える読解問題
    - Common Sense QA: 常識的な知識を必要とする多肢選択問題
    - ANLI: 2つの文の関係（含意、矛盾、中立）を判断する高度な推論問題
    - TruthfulQA: モデルが嘘や誤解を避け、どれだけ誠実に回答できるかを測定（この項目はGPT-3.5が優勢）
- 数学・コーディングベンチマーク: 数学はPhi-3-medium、コーディングはGPT-3.5が優勢
    - GSM-8K: 小学校レベルの算数の文章問題
    - MATH: 競技数学レベルの非常に高度な数学問題
    - HumanEval: Pythonのコーディング（関数作成）能力をテスト
    - MBPP: 基本的なPythonプログラミング能力をテスト
- 専門性・対話ベンチマーク: Phi-3ファミリー（Small/Medium）が優勢
    - MedQA: 医師国家試験レベルの医療知識を問う
    - GPQA: 大学院レベルの専門知識（生物学、化学、物理学）を問う
    - MT Bench: チャットボットとしての対話能力・指示追従能力を評価

長いコンテキストに対応するため、phi-3.5-miniとphi-3.5-MoEを開発:

- LongRopeとmixed context windowにより、性能を損なうことなくコンテキスト長を4Kから128Kに拡張

![](image/table1_1.png)

多言語の平均スコアは、モデルの容量の大きいphi-3.5-MoEが最も優位:

![](image/fig4.png)

RULERベンチマークの結果では、学習段階で長いデータが不足しているため性能が劣勢:

![](image/table2.png)

## Phi-3.5-Vision

Phi-3.5-Visionは、4.2Bの複数画像にとテキスト入力に対応したマルチモーダルモデル

画像エンコーダーGLIP VIT-L/14とphi-3.5-miniで構成される:

- 視覚トークンは、エンコード後にテキストトークンと交互に配置して連結
- 高解像度画像と異なるアスペクト比に対応するため、[dynamic cropping strategy][1]を採用
    - 入力画像を2次元のブロック配列に分割し、エンコード後、連結
- 複数画像の入力は、各画像のトークンを連結して処理
- 最大解像度は$1344\times 1344$

[1]: https://proceedings.neurips.cc/paper_files/paper/2024/hash/4b06cdddb1cde6624c0be1465c7b800f-Abstract-Conference.html

事前学習は、合計0.5兆トークンの多様なデータセットを用いて実施:

- [画像とテキストが交互に配置された文章データ][2]
- [FLD-5B][1]からの画像とテキストペアデータ
- PDFのOCRデータ
- 図・表・テーブルのデータ
- テキストのみのデータ

[1]: https://openaccess.thecvf.com/content/CVPR2024/html/Xiao_Florence-2_Advancing_a_Unified_Representation_for_a_Variety_of_Vision_CVPR_2024_paper.html
[2]: https://proceedings.neurips.cc/paper_files/paper/2023/hash/e2cfb719f58585f779d0a4f9f07bd618-Abstract-Datasets_and_Benchmarks.html

事後学習は、SFTとDPOの2段階で実施:

- 33BトークンのSFTのデータセット
    - 公開されているマルチモーダル指示チューニングデータセット
    - 構築した大規模なマルチモーダル指示チューニングデータセット
        - 一般的な自然画像の理解
        - 図・表・テーブル・ダイアグラムの理解・推論
        - パワーポイントの理解
        - 動画要約
        - モデルの安全性
- 小規模なDPOデータセット
- テキストのみのタスクとマルチモーダルのタスクを混合して訓練

ベンチマークは、オープンウェイトモデルに対して優勢:

![](image/table5.png)

- 科学ベンチマーク: Phi-3.5-Visionが（GPT-40を除き）圧勝
    - MMMU: 専門家レベルの超多分野マルチモーダル理解力テスト
    - Science QA: 図解などを含む科学的な質問への推論能力
    - Math Vista: グラフや図形を含む、視覚的な数学の推論能力
    - Inter-GPS: 幾何学（図形）問題の読解・推論能力
- 一般的知識ベンチマーク: Phi-3.5-Visionが非常に優秀
    - MMBench: 全方位的なマルチモーダル能力を測るベンチマーク
    - POPE: モデルが「画像に無いもの」を「ある」と答えてしまわないか（ハルシネーション）を測るテスト
- 図表・OCRベンチマーク: Phi-3.5-Visionが（GPT-40を除き）圧勝
    - AI2D: 教科書にあるような図（ダイアグラム）に関する推論問題
    - ChartQA: グラフやチャート（図表）の内容に関する質問応答
    - Text VQA: 画像の中に書かれている文字（OCR）を読み取って答える必要がある質問

複数画像・動画のベンチマークでは、オープンウェイトのモデルに匹敵する性能:

![](image/table6.png)

- BLINK: 人間が瞬時に解決できるが、AIにとって困難なタスク
- VideoMME: 動画の理解を測定するためのベンチマーク

## 実装

In [ ]:
%pip install -qU transformers==4.57.1
%pip install -qU sentencepiece protobuf bitsandbytes accelerate

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get("HF_TOKEN")
except ImportError:
    from dotenv import load_dotenv
    import os
    load_dotenv()
    HF_TOKEN = os.getenv("HF_TOKEN")

assert HF_TOKEN

import os
import logging as logging_
import transformers
import bitsandbytes
from transformers import PretrainedConfig
from transformers.utils import logging

from typing import Callable, Optional, Union

import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# トークナイザー

import os
from shutil import copyfile

from transformers import LlamaTokenizerFast, pipeline

# モデル

from typing import Callable, Optional, Union

import torch
from torch import nn
import torchvision

assert torch.cuda.is_available(), "CUDAを使用できません"

from transformers.activations import ACT2FN
from transformers.cache_utils import Cache, DynamicCache, StaticCache
from transformers.generation import GenerationMixin
from transformers.modeling_attn_mask_utils import AttentionMaskConverter, _prepare_4d_causal_attention_mask
from transformers.modeling_flash_attention_utils import is_flash_attn_available
from transformers.modeling_layers import (
    GenericForSequenceClassification,
    GradientCheckpointingLayer,
)
from transformers.modeling_outputs import MoeCausalLMOutputWithPast, MoeModelOutputWithPast
from transformers.modeling_rope_utils import ROPE_INIT_FUNCTIONS
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import auto_docstring, can_return_tuple, is_torch_flex_attn_available, logging
from transformers.utils.deprecation import deprecate_kwarg
from transformers.models.phimoe.configuration_phimoe import PhimoeConfig

if is_flash_attn_available():
    from transformers.modeling_flash_attention_utils import _flash_attention_forward

if is_torch_flex_attn_available():
    from torch.nn.attention.flex_attention import BlockMask

    from transformers.integrations.flex_attention import make_flex_block_causal_mask

# This makes `_prepare_4d_causal_attention_mask` a leaf function in the FX graph.
# It means that the function will not be traced through and simply appear as a node in the graph.
# _prepare_4d_causal_attention_mask = torch.fx.wrap(_prepare_4d_causal_attention_mask)

# デバイス設定

torch.set_default_device("cuda")

# ログ設定

if os.path.exists('debug.log'):
    os.remove('debug.log')

def custom_format(record):
    match record.levelno:
        case logging_.DEBUG:
            level = '🟦'
        case logging_.INFO:
            level = '🟩'
        case logging_.WARNING:
            level = '🟨'
        case logging_.ERROR:
            level = '🟥'
        case logging_.CRITICAL:
            level = '🛑'
    return f"{level} {record.getMessage()}"

logging.set_verbosity_debug()
logger = logging.get_logger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging_.Formatter()
formatter.format = custom_format

file_handler = logging_.FileHandler('debug.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging_.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.info(f"Transformers version: {transformers.__version__}")
logger.info(f"Numpy version: {np.__version__}")
logger.info(f"BitsAndBytes version: {bitsandbytes.__version__}")

### PhimoeRotaryEmbedding

PhimoeRoraryEmbeddingは、RoPE（Rotary Position Embedding）に必要なsinとcosを計算するクラス

In [ ]:
# Copied from transformers.models.llama.modeling_llama.rotate_half
def rotate_half(x):
    """Rotates half the hidden dims of the input."""
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids, unsqueeze_dim=1):
    """Applies Rotary Position Embedding to the query and key tensors.

    Args:
        q (`torch.Tensor`): The query tensor.
        k (`torch.Tensor`): The key tensor.
        cos (`torch.Tensor`): The cosine part of the rotary embedding.
        sin (`torch.Tensor`): The sine part of the rotary embedding.
        position_ids (`torch.Tensor`):
            The position indices of the tokens corresponding to the query and key tensors. For example, this can be
            used to pass offsetted position ids when working with a KV-cache.
        unsqueeze_dim (`int`, *optional*, defaults to 1):
            The 'unsqueeze_dim' argument specifies the dimension along which to unsqueeze cos[position_ids] and
            sin[position_ids] so that they can be properly broadcasted to the dimensions of q and k. For example, note
            that cos[position_ids] and sin[position_ids] have the shape [batch_size, seq_len, head_dim]. Then, if q and
            k have the shape [batch_size, heads, seq_len, head_dim], then setting unsqueeze_dim=1 makes
            cos[position_ids] and sin[position_ids] broadcastable to the shapes of q and k. Similarly, if q and k have
            the shape [batch_size, seq_len, heads, head_dim], then set unsqueeze_dim=2.
    Returns:
        `tuple(torch.Tensor)` comprising of the query and key tensors rotated using the Rotary Position Embedding.
    """
    cos = cos[position_ids].unsqueeze(unsqueeze_dim)
    sin = sin[position_ids].unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed

In [ ]:
class PhimoeRotaryEmbedding(nn.Module):
    def __init__(
        self,
        config: Optional[PhimoeConfig] = None,
    ):
        logger.info(f"PhimoeRotaryEmbeddingを初期化開始 {config.rope_scaling=}")
        super().__init__()

        self.config = config
        if config.rope_scaling is not None:
            # LongRopeの場合、ShortRopeとLongRopeの両方のスケーリングファクターを設定
            self.rope_type = config.rope_scaling.get("rope_type", config.rope_scaling.get("type"))

            # 1.243163
            self.short_mscale = config.rope_scaling.get("short_mscale")
            logger.debug(f"{self.short_mscale=}")

            # 1.243163
            self.long_mscale = config.rope_scaling.get("long_mscale")
            logger.debug(f"{self.long_mscale=}")
        else:
            self.rope_type = "default"

        # _compute_longrope_parameters
        self.rope_init_fn = ROPE_INIT_FUNCTIONS[self.rope_type]
        logger.debug(f"{self.rope_init_fn=}")

        logger.info(f"PhimoeRotaryEmbeddingを初期化完了")

    def forward(self, x, seq_len=None):
        logger.info(f"PhimoeRotaryEmbeddingの順伝播開始 {x.shape=} {seq_len=}")

        mscale = None

        if self.config.rope_scaling and seq_len:
            # LongRopeの場合、シーケンス長に応じてスケーリングファクターを切り替え
            # 系列長が4096以上の場合はLongRopeを使用、それ以外はShortRopeを使用
            # 130 < 4096 -> short_mscale
            mscale = (
                self.long_mscale
                if seq_len > self.config.rope_scaling["original_max_position_embeddings"]
                else self.short_mscale
            )
            logger.debug(f"{self.config.rope_scaling['original_max_position_embeddings']=}")

        # 1.243163
        logger.debug(f"{mscale=}")

        # 1. 逆周波数とアテンションスケーリングを取得
        # (64,), (1,)
        inv_freq, attention_scaling = self.rope_init_fn(self.config, x.device, seq_len)
        logger.debug(f"{inv_freq.shape=}")

        # 2. スケーリングファクターを決定
        # 1.243163
        mscale = attention_scaling if mscale is None else mscale
        logger.debug(f"{mscale=}")

        # 3. トークンの位置インデックスのリストを作成
        # (130, )
        t = torch.arange(seq_len, device=x.device, dtype=torch.float32)
        logger.debug(f"{t.shape=}")

        # 4. 位置ごとの回転角度（ラジアン）を計算
        # (130, 64)
        freqs = torch.outer(t, inv_freq)
        logger.debug(f"{freqs.shape=}")

        # 5. 2つのベクトルに対して回転させるため、回転角度を複製
        # (130, 128)
        emb = torch.cat((freqs, freqs), dim=-1)

        # 6. cosとsinを計算
        # (130, 128), (130, 128)
        res = (emb.cos() * mscale).to(x.dtype), (emb.sin() * mscale).to(x.dtype)

        logger.info(f"PhimoeRotaryEmbeddingの順伝播完了 {res[0].shape=} {res[1].shape=}")
        return res

### PhimoeAttention

PhimoeAttentionは、非効率な（eager）アテンション計算を行うクラス

In [ ]:
# Copied from transformers.models.llama.modeling_llama.repeat_kv
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    This is the equivalent of torch.repeat_interleave(x, dim=1, repeats=n_rep). The hidden states go from (batch,
    num_key_value_heads, seqlen, head_dim) to (batch, num_attention_heads, seqlen, head_dim)
    """
    logger.info(f"repeat_kvを開始 {hidden_states.shape=} {n_rep=}")
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape

    if n_rep == 1:
        return hidden_states

    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    logger.debug(f"{hidden_states.shape=}")

    res = hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)
    logger.info(f"repeat_kvを完了 {res.shape=}")
    return res


In [ ]:
class PhimoeAttention(nn.Module):
    """
    Multi-headed attention from 'Attention Is All You Need' paper. Modified to use sliding window attention: Longformer
    and "Generating Long Sequences with Sparse Transformers".
    """

    def __init__(self, config: PhimoeConfig, layer_idx: Optional[int] = None):
        logger.info(f"PhimoeAttentionを初期化開始 {layer_idx=} {config.hidden_size=} {config.num_attention_heads=} {config.num_key_value_heads=} {config.max_position_embeddings=} {config.rope_theta=} {config.attention_dropout=}")

        super().__init__()
        self.config = config

        # 0...31
        self.layer_idx = layer_idx

        if layer_idx is None:
            logger.warning_once(
                f"Instantiating {self.__class__.__name__} without passing a `layer_idx` is not recommended and will "
                "lead to errors during the forward call if caching is used. Please make sure to provide a `layer_idx` "
                "when creating this class."
            )

        # 4096
        self.hidden_size = config.hidden_size

        # 32
        self.num_heads = config.num_attention_heads

        # 4096 / 32 = 128
        self.head_dim = self.hidden_size // self.num_heads
        logger.debug(f"{self.head_dim=}")

        # 8
        self.num_key_value_heads = config.num_key_value_heads

        # 4
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        logger.debug(f"{self.num_key_value_groups=}")

        # 131072
        self.max_position_embeddings = config.max_position_embeddings

        # 10000
        self.rope_theta = config.rope_theta

        self.is_causal = True

        # 0.0
        self.attention_dropout = config.attention_dropout

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
                f" and `num_heads`: {self.num_heads})."
            )

        # 4096 -> 32 * 128 = 4096
        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=self.config.attention_bias)

        # 4096 -> 8 * 128 = 1024
        self.k_proj = nn.Linear(
            self.hidden_size, self.num_key_value_heads * self.head_dim, bias=self.config.attention_bias
        )

        # 4096 -> 8 * 128 = 1024
        self.v_proj = nn.Linear(
            self.hidden_size, self.num_key_value_heads * self.head_dim, bias=self.config.attention_bias
        )

        # 32 * 128 = 4096 -> 4096
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=self.config.attention_bias)

        logger.info(f"PhimoeAttentionを初期化完了")

    def _shape(self, tensor: torch.Tensor, seq_len: int, bsz: int):
        return tensor.view(bsz, seq_len, self.num_heads, self.head_dim).transpose(1, 2).contiguous()

    @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,
    ) -> tuple[torch.Tensor, Optional[torch.Tensor], Optional[tuple[torch.Tensor]]]:
        logger.info(f"PhimoeAttentionの順伝播開始 {hidden_state.shape=} {attention_mask.shape if attention_mask is not None else None=} {position_ids=} {past_key_values is not None=} {output_attentions=} {use_cache=} {cache_position=} {position_embeddings is not None=}")

        bsz, q_len, _ = hidden_states.size()

        # 1. クエリ・キー・バリューの生成

        query_states = self.q_proj(hidden_states)
        logger.debug(f"{query_states.shape=}")

        key_states = self.k_proj(hidden_states)
        logger.debug(f"{key_states.shape=}")

        value_states = self.v_proj(hidden_states)
        logger.debug(f"{value_states.shape=}")

        # 2. ヘッド分割とGQAの準備

        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{query_states.shape=}")

        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{key_states.shape=}")

        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{value_states.shape=}")

        # 3. RoPEの適用

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

        # 4. KVキャッシュの更新

        if past_key_values is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # Specific to RoPE models
            key_states, value_states = past_key_values.update(key_states, value_states, self.layer_idx, cache_kwargs)

        # 5. GQA用にキー・バリューをコピー

        # repeat k/v heads if n_kv_heads < n_heads
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        logger.debug(f"{key_states.shape=}")

        value_states = repeat_kv(value_states, self.num_key_value_groups)
        logger.debug(f"{value_states.shape=}")

        # 6. 生のアテンションスコアを計算

        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)
        logger.debug(f"{attn_weights.shape=}")

        # 7. 訓練時は因果マスクを適用

        if attention_mask is not None:  # no matter the length, we just slice it
            causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
            attn_weights = attn_weights + causal_mask

        # 8. アテンションスコアを計算

        # upcast attention to fp32
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)

        # 9. ドロップアウトを適用

        attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)

        # 10. アテンションを計算

        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )

        # 11. ヘッドを結合

        attn_output = attn_output.transpose(1, 2).contiguous()
        logger.debug(f"{attn_output.shape=}")

        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)
        logger.debug(f"{attn_output.shape=}")

        # 12. 出力の線形層を適用

        attn_output = self.o_proj(attn_output)
        logger.debug(f"{attn_output.shape=}")

        if not output_attentions:
            attn_weights = None

        logger.info(f"PhimoeAttentionの順伝播完了 {attn_output.shape=} {attn_weights.shape if attn_weights is not None else None=}")

        return attn_output, attn_weights

### PhimoeFlashAttention2

PhimoeFlashAttention2は、Transformersの_flash_attention_forward関数のラッパークラス

初期化関数は、PhimoeAttentionを継承

In [ ]:
class PhimoeFlashAttention2(PhimoeAttention):
    """
    Phimoe flash attention module. This module inherits from `PhimoeAttention` as the weights of the module stays
    untouched. The only required change would be on the forward pass where it needs to correctly call the public API of
    flash attention and deal with padding tokens in case the input contains any of them.
    """

    @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,
    ):
        logger.info(f"PhimoeFlashAttention2の順伝播開始 {hidden_states.shape=} {attention_mask.shape if attention_mask is not None else None=} {position_ids=} {past_key_values if past_key_values is not None else None=} {output_attentions=} {use_cache=} {cache_position=} {position_embeddings is not None=}")

        bsz, q_len, _ = hidden_states.size()

        # 1. クエリ・キー・バリューの生成

        query_states = self.q_proj(hidden_states)
        logger.debug(f"{query_states.shape=}")

        key_states = self.k_proj(hidden_states)
        logger.debug(f"{key_states.shape=}")

        value_states = self.v_proj(hidden_states)
        logger.debug(f"{value_states.shape=}")

        # 2. ヘッド分割とGQAの準備

        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{query_states.shape=}")

        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{key_states.shape=}")

        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{value_states.shape=}")

        # 3. RoPEの適用

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

        # 4. KVキャッシュの更新

        if past_key_values is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # Specific to RoPE models
            key_states, value_states = past_key_values.update(key_states, value_states, self.layer_idx, cache_kwargs)

        # 5. GQA用にキー・バリューをコピー

        # repeat k/v heads if n_kv_heads < n_heads
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        logger.debug(f"{key_states.shape=}")

        value_states = repeat_kv(value_states, self.num_key_value_groups)
        logger.debug(f"{value_states.shape=}")

        # 6. ドロップアウト率の設定

        dropout_rate = 0.0 if not self.training else self.attention_dropout
        logger.debug(f"{dropout_rate=}")

        # 7. データ型の変換

        # In PEFT, usually we cast the layer norms in float32 for training stability reasons
        # therefore the input hidden states gets silently casted in float32. Hence, we need
        # cast them back in float16 just to be sure everything works as expected.
        input_dtype = query_states.dtype
        logger.debug(f"{input_dtype=}")

        device_type = query_states.device.type if query_states.device.type != "mps" else "cpu"
        logger.debug(f"{device_type=}")

        if input_dtype == torch.float32:
            if torch.is_autocast_enabled():
                # 自動キャストが有効な場合、ターゲットデータ型を取得
                target_dtype = (
                    torch.get_autocast_dtype(device_type)
                    if hasattr(torch, "get_autocast_dtype")
                    else torch.get_autocast_gpu_dtype()
                )
            # Handle the case where the model is quantized
            elif hasattr(self.config, "_pre_quantization_dtype"):
                # モデルが量子化されている場合、事前量子化データ型を取得
                target_dtype = self.config._pre_quantization_dtype
            else:
                # それ以外はq_projの重みのデータ型をターゲットデータ型とする
                target_dtype = self.q_proj.weight.dtype

            logger.debug(f"{target_dtype=}")

            logger.warning_once(
                f"The input hidden states seems to be silently casted in float32, this might be related to"
                f" the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in"
                f" {target_dtype}."
            )

            query_states = query_states.to(target_dtype)
            key_states = key_states.to(target_dtype)
            value_states = value_states.to(target_dtype)

        # 8. 形状の変換（B, S, H, D) -> (B, H, S, D)

        # Reashape to the expected shape for Flash Attention
        query_states = query_states.transpose(1, 2)
        logger.debug(f"{query_states.shape=}")

        key_states = key_states.transpose(1, 2)
        logger.debug(f"{key_states.shape=}")

        value_states = value_states.transpose(1, 2)
        logger.debug(f"{value_states.shape=}")

        # 9. Flash Attentionを呼び出しアテンションを計算

        attn_output = _flash_attention_forward(
            query_states,
            key_states,
            value_states,
            attention_mask,
            q_len,
            position_ids=position_ids,
            dropout=dropout_rate,
            sliding_window=getattr(self.config, "sliding_window", None),
            is_causal=self.is_causal,
        )

        # 10. ヘッドを結合

        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size).contiguous()
        logger.debug(f"{attn_output.shape=}")

        # 11. 出力の線形層を適用

        attn_output = self.o_proj(attn_output)
        logger.debug(f"{attn_output.shape=}")

        if not output_attentions:
            attn_weights = None

        logger.info(f"PhimoeFlashAttention2の順伝播完了 {attn_output.shape=} {attn_weights.shape if attn_weights is not None else None=}")

        return attn_output, attn_weights

### PhimoeSdpaAttention

PhimoeSdpaAttentionは、PyTorch 2.0のscaled_dot_product_attention関数のラッパークラス

PhimoeAttentionを継承し、初期化を省略

In [ ]:
class PhimoeSdpaAttention(PhimoeAttention):
    """
    Phimoe attention module using torch.nn.functional.scaled_dot_product_attention. This module inherits from
    `PhimoeAttention` as the weights of the module stays untouched. The only changes are on the forward pass to adapt to
    SDPA API.
    """

    # Adapted from PhimoeAttention.forward
    @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,
    ) -> tuple[torch.Tensor, Optional[torch.Tensor], Optional[tuple[torch.Tensor]]]:
        logger.info(f"PhimoeSdpaAttentionの順伝播開始 {hidden_states.shape=} {attention_mask.shape if attention_mask is not None else None=} {position_ids=} {past_key_values is not None=} {output_attentions=} {use_cache=} {cache_position=} {position_embeddings is not None=}")

        if output_attentions:
            # アテンションの重みも課したい場合は、eager実装にフォールバック

            # TODO: Improve this warning with e.g. `model.config.attn_implementation = "manual"` once this is implemented.
            logger.warning_once(
                "PhimoeModel is using PhimoeSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, "
                'but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.'
            )
            return super().forward(
                hidden_states=hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                output_attentions=output_attentions,
                use_cache=use_cache,
                position_embeddings=position_embeddings,
            )

        bsz, q_len, _ = hidden_states.size()

        # 1. クエリ・キー・バリューの生成

        # (1, 130, 4096) -> (1, 130, 4096)
        query_states = self.q_proj(hidden_states)
        logger.debug(f"{query_states.shape=}")

        # (1, 130, 4096) -> (1, 130, 1024)
        key_states = self.k_proj(hidden_states)
        logger.debug(f"{key_states.shape=}")

        # (1, 130, 4096) -> (1, 130, 1024)
        value_states = self.v_proj(hidden_states)
        logger.debug(f"{value_states.shape=}")

        # 2. ヘッド分割とGQAの準備

        # (1, 130, 4096) -> (1, 32, 130, 128)
        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{query_states.shape=}")

        # (1, 130, 1024) -> (1, 8, 130, 128)
        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{key_states.shape=}")

        # (1, 130, 1024) -> (1, 8, 130, 128)
        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        logger.debug(f"{value_states.shape=}")

        # 3. RoPEの適用

        # (130, 128), (130, 128)
        cos, sin = position_embeddings

        # (1, 32, 130, 128), (1, 8, 130, 128)
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)
        logger.debug(f"{query_states.shape=}")

        # 4. KVキャッシュの更新

        if past_key_values is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # Specific to RoPE models
            key_states, value_states = past_key_values.update(key_states, value_states, self.layer_idx, cache_kwargs)

        # 5. GQA用にキー・バリューをコピー

        # (1, 8, 130, 128) -> (1, 32, 130, 128)
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        logger.debug(f"{key_states.shape=}")

        # (1, 8, 130, 128) -> (1, 32, 130, 128)
        value_states = repeat_kv(value_states, self.num_key_value_groups)
        logger.debug(f"{value_states.shape=}")

        # 6. 訓練時は因果マスクを適用

        causal_mask = attention_mask
        if attention_mask is not None:  # no matter the length, we just slice it
            causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]

        # 7. クエリ・キー・バリューのメモリを連続化（バグ回避）

        # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
        # Reference: https://github.com/pytorch/pytorch/issues/112577.
        if query_states.device.type == "cuda" and attention_mask is not None:
            query_states = query_states.contiguous()
            key_states = key_states.contiguous()
            value_states = value_states.contiguous()

        # 8. is_causalフラグを設定

        # We dispatch to SDPA's Flash Attention or Efficient kernels via this `is_causal` if statement instead of an inline conditional assignment
        # in SDPA to support both torch.compile's dynamic shapes and full graph options. An inline conditional prevents dynamic shapes from compiling.
        # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
        is_causal = causal_mask is None and q_len > 1

        # 9. SDPAを呼び出しアテンションを計算

        # (1, 32, 130, 128), (1, 32, 130, 128), (1, 32, 130, 128) -> (1, 32, 130, 128)
        attn_output = torch.nn.functional.scaled_dot_product_attention(
            query_states,
            key_states,
            value_states,
            attn_mask=causal_mask,
            dropout_p=self.attention_dropout if self.training else 0.0,
            is_causal=is_causal,
        )
        logger.debug(f"{attn_output.shape=}")

        # 10. ヘッドを結合

        # (1, 32, 130, 128) -> (1, 130, 4096)
        attn_output = attn_output.transpose(1, 2).contiguous()
        logger.debug(f"{attn_output.shape=}")

        # (1, 130, 4096) -> (1, 130, 4096)
        attn_output = attn_output.view(bsz, q_len, self.hidden_size)
        logger.debug(f"{attn_output.shape=}")

        # 11. 出力の線形層を適用

        # (1, 130, 4096) -> (1, 130, 4096)
        attn_output = self.o_proj(attn_output)
        logger.debug(f"{attn_output.shape=}")

        logger.info(f"PhimoeSdpaAttentionの順伝播完了 {attn_output.shape=}")

        return attn_output, None

In [ ]:
PHIMOE_ATTENTION_CLASSES = {
    "eager": PhimoeAttention,
    "flash_attention_2": PhimoeFlashAttention2,
    "sdpa": PhimoeSdpaAttention,
}

### PhimoeBlackSparseTop2MLP

PhimoeBlackSparseTop2MLPは、単一のエキスパートネットワークの実装クラス

MixtralのMLPアーキテクチャをコピーし、SwiGLUを採用

In [ ]:
# Copied from transformers.models.mixtral.modeling_mixtral.MixtralBlockSparseTop2MLP with Mixtral->Phimoe
class PhimoeBlockSparseTop2MLP(nn.Module):
    def __init__(self, config: PhimoeConfig):
        logger.info(f"PhimoeBlockSparseTop2MLPを初期化開始 {config.hidden_size=} {config.intermediate_size=} {config.hidden_act=}")

        super().__init__()

        # 6400
        self.ffn_dim = config.intermediate_size

        # 4096
        self.hidden_dim = config.hidden_size

        # 4096 -> 6400
        self.w1 = nn.Linear(self.hidden_dim, self.ffn_dim, bias=False)

        # 6400 -> 4096
        self.w2 = nn.Linear(self.ffn_dim, self.hidden_dim, bias=False)

        # 4096 -> 6400
        self.w3 = nn.Linear(self.hidden_dim, self.ffn_dim, bias=False)

        # SiLU
        self.act_fn = ACT2FN[config.hidden_act]
        logger.debug(f"{self.act_fn=}")

        logger.info(f"PhimoeBlockSparseTop2MLPを初期化完了")

    def forward(self, hidden_states):
        logger.info(f"PhimoeBlockSparseTop2MLPの順伝播開始 {hidden_states.shape=}")

        # current_hidden_states = self.act_fn(self.w1(hidden_states)) * self.w3(hidden_states)

        # 1. ゲートを計算
        gate = self.act_fn(self.w1(hidden_states))
        logger.debug(f"{gate.shape=}")

        # 2. アッププロジェクションを計算
        up = self.w3(hidden_states)
        logger.debug(f"{up.shape=}")


        # 3. ゲートを適用し、ダウンプロジェクションを計算
        # current_hidden_states = self.w2(current_hidden_states)
        current_hidden_states = self.w2(gate * up)
        logger.debug(f"{current_hidden_states.shape=}")

        logger.info(f"PhimoeBlockSparseTop2MLPの順伝播完了 {current_hidden_states.shape=}")
        return current_hidden_states

### MultiplierProcessor

MultiplierProcessorは、学習可能なルーティングネットワークの勾配を計算するクラス

「エキスパートを確率的に選ぶ」という離散的な演算の誤差逆伝播を、Heunの3次法で勾配をより正確に推定する

訓練次のみ使用

![](image/algorithm1.png)

誤差逆伝搬では、出力からの勾配とSoftmaxの微分の積を計算している:

$$
\hat{\nabla}_{D,\text{SparseMixer-v2}}z \approx f'(\dots) \cdot \frac{\partial p_D}{\partial z}
$$

In [ ]:
class MultiplierProcessor(torch.autograd.Function):
    @staticmethod
    def forward(
        ctx,
        scores: torch.Tensor,
        multiplier: torch.Tensor,
        selected_experts: torch.Tensor,
        masked_gates: torch.Tensor,
        mask_for_one: torch.Tensor,
    ):
        """
        Forward pass for the custom autograd function.

        Args:
            ctx: Context object to save information for backward computation.
            scores (torch.Tensor): Input scores tensor.
            multiplier (torch.Tensor): Multiplier tensor.
            selected_experts (torch.Tensor): Tensor of selected experts.
            masked_gates (torch.Tensor): Masked gates tensor.
            mask_for_one (torch.Tensor): Mask for one tensor.

        Returns:
            torch.Tensor: Result of the forward pass.
        """
        logger.info(f"MultiplierProcessorの順伝播開始 {scores.shape=} {multiplier.shape=} {selected_experts.shape=} {masked_gates.shape=} {mask_for_one.shape=}")

        # 誤差逆伝播用にテンソルを保存
        ctx.save_for_backward(multiplier, selected_experts, masked_gates)

        # Sparsemixerから受け取ったエキスパートの重みに対し、Heunの3次法に基づく係数を適用
        res = multiplier * mask_for_one

        # 調整済みのエキスパートの重みを返す
        logger.info(f"MultiplierProcessorの順伝播完了 {res.shape=}")
        return res

    @staticmethod
    def backward(
        ctx,
        grad_at_output: torch.Tensor,
    ):
        """
        Backward pass for the custom autograd function.

        Args:
            ctx: Context object with saved tensors from the forward pass.
            grad_at_output (torch.Tensor): Gradient at the output.

        Returns:
            tuple[torch.Tensor, None, None, None, None]: Gradients for the inputs.
        """
        logger.info(f"MultiplierProcessorの逆伝播開始 {grad_at_output.shape=}")

        # 1. テンソルを復元

        # multiplier: 重み, selected_experts: 選択されたエキスパート, masked_gates: softmax後のスコア
        multiplier, selected_experts, masked_gates = ctx.saved_tensors

        # 2. 出力側の勾配を調整

        # 出力側の勾配 grad_at_output に元の重み p_D を乗算（v2の式8）
        grad_at_output = grad_at_output * multiplier

        # 3. Softmax関数の微分を手動で計算
        grad_at_scores_expanded = masked_gates * grad_at_output.mul(-1)
        grad_at_scores_expanded.scatter_add_(
            dim=-1,
            index=selected_experts,
            src=grad_at_output,
        )

        logger.info(f"MultiplierProcessorの逆伝播完了 {grad_at_scores_expanded.shape=}")

        # 4. 訓練対象のテンソルの勾配を返す

        return (
            grad_at_scores_expanded,
            None,
            None,
            None,
            None,
        )

Sparsemixer関数は、トークンを処理する上位2つのエキスパートを選択し、その重みを計算するルーティング関数

訓練を安定化させ、効率的に勾配計算が可能

TopKを一つずつ処理:

![](image/algorithm3.png)

In [ ]:
def sparsemixer(scores, jitter_eps, training, top_k=2):
    """
    Sparse mixer function to select top-k experts and compute multipliers.
    Based on the paper: https://huggingface.co/papers/2409.12136
    We first replace the TopK(·) function as random sampling of discrete variables
    in model training. Then, following Liu et al. (2023a) and Liu et al. (2023b), we apply Heun's
    third order method to approximate the expert routing gradient and construct a modified
    back-propagation to give a mathematically sound gradient estimation for expert routing.

    Args:
        scores (torch.Tensor): Input scores tensor.
        jitter_eps (float): Jitter epsilon for numerical stability.
        training (bool): Flag indicating if the model is in training mode.
        top_k (int): Number of top experts to select.

    Returns:
        tuple[torch.Tensor, torch.Tensor]: Multiplier and selected experts tensors.
    """
    logger.info(f"sparsemixerを開始 {scores.shape=} {jitter_eps=} {training=} {top_k=}")


    if top_k != 2:
        raise ValueError("top_k must be equal to 2")

    # first expert

    # 1. スコアが明らかに低いエキスパートを除外するためのマスクを作成（MaskedSoftmax）

    with torch.no_grad():
        # 最大スコアを取得
        mask_logits_threshold, max_ind = scores.max(dim=-1, keepdim=True)
        # スコアの絶対値を使用して正規化ファクターを計算
        factor = scores.abs().clamp(min=mask_logits_threshold)
        # マスクを計算 (最大スコア - 各スコア) / 正規化ファクター > 2 * 微小値
        mask_logits_threshold = ((mask_logits_threshold - scores) / factor) > (2 * jitter_eps)

    # 2. スコアにマスクを適用（スパース化）

    # Apply mask
    # (130, 16) -> (130, 16)
    masked_gates = scores.masked_fill(mask_logits_threshold, float("-inf"))

    # 3. 1つ目のエキスパートを選択（Sample D from p）

    if training:
        # Gumbelサンプリングを使用して確率的に1つ目のエキスパートを選択
        # Gumbelサンプリングは、確率的かつ離散的な選択を微分可能な演算に置き換えるテクニック
        # 具体的には、マスクしたスコアに対してGumbelノイズを加え、最大値のインデックスを取得する
        selected_experts = (
            (
                masked_gates
                - torch.empty_like(masked_gates, memory_format=torch.legacy_contiguous_format).exponential_().log()
            )
            .max(dim=-1)[1]
            .unsqueeze(-1)
        )  # Gumbel sampling, more robust than the multinomial method
    else:
        # 推論時は最大スコアのエキスパートを選択
        selected_experts = max_ind

    # 4. ソフトマックスを適用し、選択したエキスパートの重みを取得（h <- Expert(x, W_d）p_D)

    # Compute scores for gradients
    masked_gates = torch.softmax(masked_gates, dim=-1)

    # (130, 16) -> (130, 1)
    multiplier_o = masked_gates.gather(dim=-1, index=selected_experts)

    if training:
        # Heunの3次法に基づく勾配調整用のマスク（mask_for_one）を計算

        # 決定論的に最もスコアが高いエキスパートのインデックスを取得
        max_scores, max_ind = masked_gates.max(dim=-1, keepdim=True)

        # 以下のどちらかの条件が満たされた場合にTrueとなるマスクを作成
        mask_for_one = torch.logical_or(
            selected_experts == max_ind, # A. Gumbelサンプルがたまたま決定論的な最良のエキスパートを選択した場合（\delta_D == 1）
            torch.rand_like(max_scores) > 0.75, # B. 25%の確率でTrue（Bournoulli(1/4)）
        )

        # mask_for_oneの値を計算（max(\delta_D, (1+2B) / 3）
        # Trueの場合: 0.3333 + 1.0 * 0.6667 = 1.0
        # Falseの場合: 0.3333 + 0.0 * 0.6667 = 0.3333 = 1/3
        mask_for_one = torch.add(0.3333, mask_for_one, alpha=0.6667).type_as(masked_gates)

        # mask_for_oneをカスタム勾配計算関数に渡す
        # 順伝播: 勾配調整用のマスクmask_for_oneを元のエキスパートの重みmultiplierに乗算する
        # 逆伝播: 調整された重みを使って勾配を計算する
        # y <- h + detach(mask_for_one * h - h)
        multiplier = MultiplierProcessor.apply(
            scores,
            multiplier_o,
            selected_experts,
            masked_gates,
            mask_for_one,
        )
    else:
        # 推論時は何もしない
        multiplier = multiplier_o

    # 5. 最初に選択したエキスパートを除外

    # Masked out first expert
    masked_scores = torch.scatter(
        scores,
        -1,
        selected_experts,
        float("-inf"),
    )

    # 6. 2つ目のエキスパートを選択

    with torch.no_grad():
        # Compute mask for sparsity
        # 最大スコアを取得
        mask_logits_threshold, max_ind = masked_scores.max(dim=-1, keepdim=True)
        # スコアの絶対値を使用して正規化ファクターを計算
        factor = scores.abs().clamp(min=mask_logits_threshold)
        # マスクを計算 (最大スコア - 各スコア) / 正規化ファクター > 2 * 微小値
        mask_logits_threshold = ((mask_logits_threshold - scores) / factor) > (2 * jitter_eps)

    # Apply mask
    masked_gates_top2 = masked_scores.masked_fill(mask_logits_threshold, float("-inf"))

    if training:
        # Gumbelサンプリングを使用して確率的に2つ目のエキスパートを選択
        selected_experts_top2 = (
            (
                masked_gates_top2
                - torch.empty_like(masked_gates_top2, memory_format=torch.legacy_contiguous_format)
                .exponential_()
                .log()
            )
            .max(dim=-1)[1]
            .unsqueeze(-1)
        )  # Gumbel sampling, more robust than the multinomial method
    else:
        # 推論時は最大スコアのエキスパートを選択
        selected_experts_top2 = max_ind

    # 7. ソフトマックスを適用し、選択したエキスパートの重みを取得

    # Compute scores for gradients
    masked_gates_top2 = torch.softmax(masked_gates_top2, dim=-1)

    # (130, 16) -> (130, 1)
    multiplier_top2_o = masked_gates_top2.gather(dim=-1, index=selected_experts_top2)

    if training:
        # Heunの3次法に基づく勾配調整用のマスクを計算
        # Compute midpoint mask
        max_scores, max_ind = masked_gates_top2.max(dim=-1, keepdim=True)
        mask_for_one_top2 = torch.logical_or(
            selected_experts_top2 == max_ind,
            torch.rand_like(max_scores).uniform_() > 0.75,  # Heun's third-order method
        )
        # 1 -> 1.0 & 0 -> 1./3: lambda x: (x + 0.5) / 1.5
        mask_for_one_top2 = torch.add(0.3333, mask_for_one_top2, alpha=0.6667).type_as(masked_gates_top2)

        # mask_for_one_top2をカスタム勾配計算関数に渡す
        # 順伝播: 勾配調整用のマスクmask_for_one_top2を元のエキスパートの重みmultiplier_top2に乗算する
        # 逆伝播: 調整された重みを使って勾配を計算する
        multiplier_top2 = MultiplierProcessor.apply(
            scores,
            multiplier_top2_o,
            selected_experts_top2,
            masked_gates_top2,
            mask_for_one_top2,
        )
    else:
        # 推論時は何もしない
        multiplier_top2 = multiplier_top2_o

    # 8. 1つ目と2つ目のエキスパートと重みとインデックスを返す

    # (130, 2)
    multiplier = torch.concat((multiplier, multiplier_top2), dim=-1)

    # (130, 2)
    selected_experts = torch.concat((selected_experts, selected_experts_top2), dim=-1)

    logger.info(f"sparsemixerを完了 {multiplier.shape=} {selected_experts.shape=}")

    return (
        multiplier,
        selected_experts,
    )

### PhimoeSparseMoeBlock

PhimoeSparseMoeBlockは、PhimoeモデルのTransformerに組み込まれているMoE層

In [ ]:
class PhimoeSparseMoeBlock(nn.Module):
    """
    This implementation is
    strictly equivalent to standard MoE with full capacity (no
    dropped tokens). It's faster since it formulates MoE operations
    in terms of block-sparse operations to accommodate imbalanced
    assignments of tokens to experts, whereas standard MoE either
    (1) drop tokens at the cost of reduced performance or (2) set
    capacity factor to number of experts and thus waste computation
    and memory on padding.
    """

    def __init__(self, config):
        logger.info(f"PhimoeSparseMoeBlockを初期化開始 {config.hidden_size=} {config.intermediate_size=} {config.num_local_experts=} {config.num_experts_per_tok=} {config.router_jitter_noise=} {config.input_jitter_noise=}")

        super().__init__()

        # 4096
        self.hidden_dim = config.hidden_size

        # 6400
        self.ffn_dim = config.intermediate_size

        # 16
        self.num_experts = config.num_local_experts

        # 2
        self.top_k = config.num_experts_per_tok

        # ルーターネットワーク
        # 4096 -> 16
        self.gate = nn.Linear(self.hidden_dim, self.num_experts, bias=False)

        # 複数のエキスパートネットワーク
        # 16個のエキスパート
        self.experts = nn.ModuleList([PhimoeBlockSparseTop2MLP(config) for _ in range(self.num_experts)])

        # 訓練時に使用するルーター用のノイズ
        # 0.01
        self.router_jitter_noise = config.router_jitter_noise

        # 訓練時に使用する入力用のノイズ
        # 0.01
        self.input_jitter_noise = config.input_jitter_noise
        logger.info(f"PhimoeSparseMoeBlockを初期化完了")

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        logger.info(f"PhimoeSparseMoeBlockの順伝播開始 {hidden_states.shape=}")

        batch_size, sequence_length, hidden_dim = hidden_states.shape

        if self.training and self.input_jitter_noise > 0:
            # 訓練時に入力にノイズを加える（安定化のため）
            hidden_states *= torch.empty_like(hidden_states).uniform_(
                1.0 - self.input_jitter_noise, 1.0 + self.input_jitter_noise
            )

        # （バッチサイズ*シーケンス長, 隠れ層次元数)の2次元に変換
        # (1, 130, 4096) -> (130, 4096)
        hidden_states = hidden_states.view(-1, hidden_dim)
        logger.debug(f"{hidden_states.shape=}")

        # 1. ルータースコアを計算（各エキスパートに割り当てる確率）

        # (130, 4096) -> (130, 16)
        router_logits = self.gate(hidden_states)
        logger.debug(f"{router_logits.shape=}")

        # 2. sparsemixerを使用してトップ2のエキスパートの重みとインデックスを取得

        # (130, 16) -> (130, 2), (130, 2)
        routing_weights, selected_experts = sparsemixer(
            router_logits,
            jitter_eps=self.router_jitter_noise,
            training=self.training,
        )
        logger.debug(f"{routing_weights.shape=} {selected_experts.shape=}")

        # 3. 最終的な隠れ状態をゼロで初期化

        # (130, 4096)
        final_hidden_states = torch.zeros(
            (batch_size * sequence_length, hidden_dim), dtype=hidden_states.dtype, device=hidden_states.device
        )
        logger.debug(f"{final_hidden_states.shape=}")

        # 4. one hotエンコーディングを使用してエキスパートマスクを作成

        # どのエキスパートが、どのトークンの何番目の選択（トップ1 or トップ2）に対応しているかを示すマスク
        # (エキスパート数, TopK, トークン数)
        # (16, 2, 130)
        # One hot encode the selected experts to create an expert mask
        # this will be used to easily index which expert is going to be sollicitated
        expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=self.num_experts).permute(2, 1, 0)
        logger.debug(f"{expert_mask.shape=}")

        # 5. エキスパートごとに一度の順伝播で計算を実行（合計16回）

        # Loop over all available experts in the model and perform the computation on each expert
        for expert_idx in range(self.num_experts):
            logger.debug(f"エキスパート {expert_idx+1}/{self.num_experts} の処理開始")

            expert_layer = self.experts[expert_idx]

            # トークンを特定（トップKのインデックスidx（0or1）とトークンのインデックスtop_x（0〜129））
            idx, top_x = torch.where(expert_mask[expert_idx])
            logger.debug(f"{idx=}, {top_x=}")

            if top_x.shape[0] == 0:
                logger.debug(f"エキスパート {expert_idx+1}/{self.num_experts} の処理スキップ")
                continue

            # エキスパートが処理するトークンのベクトルを抽出
            # (n, 4096)
            # Index the correct hidden states and compute the expert hidden state for
            # the current expert. We need to make sure to multiply the output hidden
            # states by `routing_weights` on the corresponding tokens (top-1 and top-2)
            current_state = hidden_states[None, top_x].reshape(-1, hidden_dim)
            logger.debug(f"{current_state.shape=}")

            # 順伝播を実行し、ルーティング重みを適用
            # (n, 4096)
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]

            # 最終的な隠れ状態に加算
            # (130, 4096)
            # However `index_add_` only support torch tensors for indexing so we'll use
            # the `top_x` tensor here.
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))

            logger.debug(f"エキスパート {expert_idx+1}/{self.num_experts} の処理完了")

        # 6. 元の形状に戻す

        # (130, 4096) -> (1, 130, 4096)
        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)
        logger.debug(f"{final_hidden_states.shape=}")

        # (1, 130, 4096), (130, 2)
        logger.info(f"PhimoeSparseMoeBlockの順伝播完了 {final_hidden_states.shape=} {router_logits.shape=}")
        return final_hidden_states, router_logits

### PhimoeDecoderLayer

PhimoeDecoderLayerは、Phiemoeモデルを構成するTransformerデコーダー層

In [ ]:
class PhimoeDecoderLayer(GradientCheckpointingLayer):
    def __init__(self, config: PhimoeConfig, layer_idx: int):
        logger.info(f"PhimoeDecoderLayerを初期化開始 {config.hidden_size=} {config._attn_implementation=} {layer_idx=}")

        super().__init__()

        # 4096
        self.hidden_size = config.hidden_size

        # SDPA
        self.self_attn = PHIMOE_ATTENTION_CLASSES[config._attn_implementation](config, layer_idx)
        logger.debug(f"{self.self_attn=}")

        self.block_sparse_moe = PhimoeSparseMoeBlock(config)

        # 4096 -> 4096
        self.input_layernorm = nn.LayerNorm(config.hidden_size, eps=config.rms_norm_eps, elementwise_affine=True)

        # 4096 -> 4096
        self.post_attention_layernorm = nn.LayerNorm(
            config.hidden_size, eps=config.rms_norm_eps, elementwise_affine=True
        )

        logger.info(f"PhimoeDecoderLayerを初期化完了")

    @deprecate_kwarg("past_key_value", new_name="past_key_values", version="4.58")
    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        output_attentions: Optional[bool] = False,
        output_router_logits: Optional[bool] = False,
        use_cache: Optional[bool] = False,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[tuple[torch.Tensor, torch.Tensor]] = None,
        **kwargs,
    ) -> tuple[torch.FloatTensor, Optional[tuple[torch.FloatTensor, torch.FloatTensor]]]:
        """
        Args:
            hidden_states (`torch.FloatTensor`): input to the layer of shape `(batch, seq_len, embed_dim)`
            attention_mask (`torch.FloatTensor`, *optional*): attention mask of size
                `(batch, sequence_length)` where padding elements are indicated by 0.
            past_key_values (`Cache`, *optional*): cached past key and value projection states
            output_attentions (`bool`, *optional*):
                Whether or not to return the attentions tensors of all attention layers. See `attentions` under
                returned tensors for more detail.
            output_router_logits (`bool`, *optional*):
                Whether or not to return the logits of all the routers. They are useful for computing the router loss, and
                should not be returned during inference.
            use_cache (`bool`, *optional*):
                If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding
                (see `past_key_values`).
            cache_position (`torch.LongTensor` of shape `(sequence_length)`, *optional*):
                Indices depicting the position of the input sequence tokens in the sequence.
            kwargs (`dict`, *optional*):
                Arbitrary kwargs to be ignored, used for FSDP and other methods that injects code
                into the model
        """
        logger.info(f"PhimoeDecoderLayerの順伝播開始 {hidden_states.shape=} {attention_mask.shape if attention_mask is not None else None=} {position_ids=} {past_key_values is not None=} {output_attentions=} {output_router_logits=} {use_cache=} {cache_position=} {position_embeddings is not None=}")

        residual = hidden_states

        # 1. 入力のレイヤー正規化

        # (1, 130, 4096) -> (1, 130, 4096)
        hidden_states = self.input_layernorm(hidden_states)
        logger.debug(f"{hidden_states.shape=}")

        # 2. アテンションの計算

        # Self Attention
        # (1, 130, 4096) -> (1, 130, 4096)
        hidden_states, self_attn_weights = self.self_attn(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            output_attentions=output_attentions,
            use_cache=use_cache,
            cache_position=cache_position,
            position_embeddings=position_embeddings,
        )

        # 3. 残差接続

        # (1, 130, 4096) + (1, 130, 4096) -> (1, 130, 4096)
        hidden_states = residual + hidden_states

        # Fully Connected
        residual = hidden_states

        # 4. ポストアテンションのレイヤー正規化

        # (1, 130, 4096) -> (1, 130, 4096)
        hidden_states = self.post_attention_layernorm(hidden_states)

        # 5. ブロックスパースMoEの計算

        # (1, 130, 4096) -> (1, 130, 4096), (130, 16)
        hidden_states, router_logits = self.block_sparse_moe(hidden_states)

        # 6. 残差接続

        # (1, 130, 4096) + (1, 130, 4096) -> (1, 130, 4096)
        hidden_states = residual + hidden_states

        outputs = (hidden_states,)

        if output_attentions:
            outputs += (self_attn_weights,)

        if output_router_logits:
            outputs += (router_logits,)

        logger.info(f"PhimoeDecoderLayerの順伝播完了 {hidden_states.shape=} {self_attn_weights.shape if output_attentions else None=} {router_logits.shape if output_router_logits else None=}")
        return outputs

### PhimoePreTrainedModel

PhimoePreTrainedModelは、事前学習済みモデルの基本機能を継承したPhiemoeModelの基盤クラス

In [ ]:
class PhimoePreTrainedModel(PreTrainedModel):
    config: PhimoeConfig
    base_model_prefix = "model"
    supports_gradient_checkpointing = True
    _no_split_modules = ["PhimoeDecoderLayer"]
    _skip_keys_device_placement = ["past_key_values"]
    _supports_flash_attn = True
    _supports_sdpa = True

    _can_compile_fullgraph = False  # MoE models don't work with torch.compile (`torch.where(condition)` not supported)

    def _init_weights(self, module):
        std = self.config.initializer_range
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=std)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)


### PhimoeModel

PhimoeModelは、アーキテクチャの本体を構成するクラス

In [ ]:
class PhimoeModel(PhimoePreTrainedModel):
    """
    Transformer decoder consisting of *config.num_hidden_layers* layers. Each layer is a [`PhimoeDecoderLayer`]
    Args:
        config: PhimoeConfig
    """

    def __init__(self, config: PhimoeConfig):
        logger.info(f"PhimoeModelを初期化開始 {config.vocab_size=} {config.hidden_size=} {config.num_hidden_layers=} {config._attn_implementation=}")
        super().__init__(config)
        self.padding_idx = config.pad_token_id

        # 32064
        self.vocab_size = config.vocab_size

        # 32064 -> 4096
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)

        # 32層のデコーダー層
        self.layers = nn.ModuleList(
            [PhimoeDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )

        # SDPA
        self._attn_implementation = config._attn_implementation

        # 4096 -> 4096
        self.norm = nn.LayerNorm(config.hidden_size, eps=config.rms_norm_eps, elementwise_affine=True)

        self.rotary_emb = PhimoeRotaryEmbedding(config=config)

        self.gradient_checkpointing = False
        # Initialize weights and apply final processing
        self.post_init()
        logger.info(f"PhimoeModelを初期化完了")

    @can_return_tuple
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        output_router_logits: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
    ) -> MoeModelOutputWithPast:
        logger.info(f"PhimoeModelの順伝播開始 {input_ids.shape if input_ids is not None else None=} {attention_mask.shape if attention_mask is not None else None=} {position_ids=} {past_key_values is not None=} {inputs_embeds.shape if inputs_embeds is not None else None=} {use_cache=} {output_attentions=} {output_hidden_states=} {output_router_logits=} {cache_position=}")

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_router_logits = (
            output_router_logits if output_router_logits is not None else self.config.output_router_logits
        )
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache

        if (input_ids is None) ^ (inputs_embeds is not None):
            raise ValueError(
                "You cannot specify both input_ids and inputs_embeds at the same time, and must specify either one"
            )

        if self.gradient_checkpointing and self.training:
            if use_cache:
                logger.warning_once(
                    "`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`..."
                )
                use_cache = False

        if use_cache and past_key_values is None:
            past_key_values = DynamicCache(config=self.config)

        # 1. トークンを埋め込みベクトルに変換

        if inputs_embeds is None:
            # (1, 130, 32064) -> (1, 130, 4096)
            inputs_embeds = self.embed_tokens(input_ids)
            logger.debug(f"{inputs_embeds.shape=}")

        if cache_position is None:
            past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
            cache_position = torch.arange(
                past_seen_tokens, past_seen_tokens + inputs_embeds.shape[1], device=inputs_embeds.device
            )
        if position_ids is None:
            position_ids = cache_position.unsqueeze(0)

        # 2. 訓練時はアテンションマスクを作成

        causal_mask = self._update_causal_mask(
            attention_mask, inputs_embeds, cache_position, past_key_values, output_attentions
        )

        hidden_states = inputs_embeds

        # 3. RoPEのsinとcosを計算

        position_embeddings = self.rotary_emb(hidden_states, seq_len=cache_position[-1] + 1)

        # 4. デコーダー層を順番に適用

        # decoder layers
        all_hidden_states = () if output_hidden_states else None
        all_self_attns = () if output_attentions else None
        all_router_logits = () if output_router_logits else None

        # 32層のデコーダー層を順番に適用
        for decoder_layer in self.layers:
            if output_hidden_states:
                all_hidden_states += (hidden_states,)

            # (1, 130, 4096) -> (1, 130, 4096)
            layer_outputs = decoder_layer(
                hidden_states,
                attention_mask=causal_mask,
                position_ids=position_ids,
                past_key_values=past_key_values,
                output_attentions=output_attentions,
                output_router_logits=output_router_logits,
                use_cache=use_cache,
                cache_position=cache_position,
                position_embeddings=position_embeddings,
            )

            hidden_states = layer_outputs[0]

            if output_attentions:
                all_self_attns += (layer_outputs[1],)

            if output_router_logits:
                all_router_logits += (layer_outputs[-1],)

        # 5. 最終レイヤー正規化

        # (1, 130, 4096) -> (1, 130, 4096)
        hidden_states = self.norm(hidden_states)

        # add hidden states from the last decoder layer
        if output_hidden_states:
            all_hidden_states += (hidden_states,)

        logger.info(f"PhimoeModelの順伝播完了 {hidden_states.shape=} {len(all_hidden_states) if all_hidden_states is not None else None=} {len(all_self_attns) if all_self_attns is not None else None=} {len(all_router_logits) if all_router_logits is not None else None=}")

        return MoeModelOutputWithPast(
            last_hidden_state=hidden_states,
            past_key_values=past_key_values,
            hidden_states=all_hidden_states,
            attentions=all_self_attns,
            router_logits=all_router_logits,
        )

    def _update_causal_mask(
        self,
        attention_mask: Union[torch.Tensor, "BlockMask"],
        input_tensor: torch.Tensor,
        cache_position: torch.Tensor,
        past_key_values: Cache,
        output_attentions: bool = False,
    ):
        if self.config._attn_implementation == "flash_attention_2":
            if attention_mask is not None and past_key_values is not None:
                is_padding_right = attention_mask[:, -1].sum().item() != input_tensor.size()[0]
                if is_padding_right:
                    raise ValueError(
                        "You are attempting to perform batched generation with padding_side='right'"
                        " this may lead to unexpected behaviour for Flash Attention version of Phimoe. Make sure to "
                        " call `tokenizer.padding_side  = 'left'` before tokenizing the input. "
                    )
            if attention_mask is not None and 0.0 in attention_mask:
                return attention_mask
            return None
        if self.config._attn_implementation == "flex_attention":
            if isinstance(attention_mask, torch.Tensor):
                attention_mask = make_flex_block_causal_mask(attention_mask)
            return attention_mask

        # For SDPA, when possible, we will rely on its `is_causal` argument instead of its `attn_mask` argument, in
        # order to dispatch on Flash Attention 2. This feature is not compatible with static cache, as SDPA will fail
        # to infer the attention mask.
        past_seen_tokens = past_key_values.get_seq_length() if past_key_values is not None else 0
        using_static_cache = isinstance(past_key_values, StaticCache)

        # When output attentions is True, sdpa implementation's forward method calls the eager implementation's forward
        if self.config._attn_implementation == "sdpa" and not using_static_cache and not output_attentions:
            if AttentionMaskConverter._ignore_causal_mask_sdpa(
                attention_mask,
                inputs_embeds=input_tensor,
                past_key_values_length=past_seen_tokens,
                sliding_window=self.config.sliding_window,
                is_training=self.training,
            ):
                return None

        dtype = input_tensor.dtype
        min_dtype = torch.finfo(dtype).min
        sequence_length = input_tensor.shape[1]
        # StaticCache
        if using_static_cache:
            target_length = past_key_values.get_max_cache_shape()
        # DynamicCache or no cache
        else:
            target_length = (
                attention_mask.shape[-1]
                if isinstance(attention_mask, torch.Tensor)
                else past_seen_tokens + sequence_length + 1
            )

        # In case the provided `attention` mask is 2D, we generate a causal mask here (4D).
        causal_mask = self._prepare_4d_causal_attention_mask_with_cache_position(
            attention_mask,
            sequence_length=sequence_length,
            target_length=target_length,
            dtype=dtype,
            cache_position=cache_position,
            batch_size=input_tensor.shape[0],
            config=self.config,
            past_key_values=past_key_values,
        )

        if (
            self.config._attn_implementation == "sdpa"
            and attention_mask is not None
            and attention_mask.device.type in ["cuda", "xpu", "npu"]
            and not output_attentions
        ):
            # Attend to all tokens in fully masked rows in the causal_mask, for example the relevant first rows when
            # using left padding. This is required by F.scaled_dot_product_attention memory-efficient attention path.
            # Details: https://github.com/pytorch/pytorch/issues/110213
            causal_mask = AttentionMaskConverter._unmask_unattended(causal_mask, min_dtype)

        return causal_mask

    @staticmethod
    def _prepare_4d_causal_attention_mask_with_cache_position(
        attention_mask: torch.Tensor,
        sequence_length: int,
        target_length: int,
        dtype: torch.dtype,
        cache_position: torch.Tensor,
        batch_size: int,
        config: PhimoeConfig,
        past_key_values: Cache,
    ):
        """
        Creates a causal 4D mask of shape `(batch_size, 1, query_length, key_value_length)` from a 2D mask of shape
        `(batch_size, key_value_length)`, or if the input `attention_mask` is already 4D, do nothing.

        Args:
            attention_mask (`torch.Tensor`):
                A 2D attention mask of shape `(batch_size, key_value_length)` or a 4D attention mask of shape `(batch_size, 1, query_length, key_value_length)`.
            sequence_length (`int`):
                The sequence length being processed.
            target_length (`int`):
                The target length: when generating with static cache, the mask should be as long as the static cache, to account for the 0 padding, the part of the cache that is not filled yet.
            dtype (`torch.dtype`):
                The dtype to use for the 4D attention mask.
            cache_position (`torch.Tensor`):
                Indices depicting the position of the input sequence tokens in the sequence.
            batch_size (`torch.Tensor`):
                Batch size.
            config (`PhimoeConfig`):
                The model's configuration class
            past_key_values (`Cache`):
                The cache class that is being used currently to generate
        """
        if attention_mask is not None and attention_mask.dim() == 4:
            # In this case we assume that the mask comes already in inverted form and requires no inversion or slicing.
            causal_mask = attention_mask
        else:
            min_dtype = torch.finfo(dtype).min
            causal_mask = torch.full(
                (sequence_length, target_length), fill_value=min_dtype, dtype=dtype, device=cache_position.device
            )
            diagonal_attend_mask = torch.arange(target_length, device=cache_position.device) > cache_position.reshape(
                -1, 1
            )
            text_config = config.get_text_config()
            if getattr(text_config, "use_sliding_window", True) and text_config.sliding_window is not None:
                # if we have sliding window, we should not attend to tokens beyond sliding window length, so we mask them out also
                # the check is needed to verify is current checkpoint was trained with sliding window or not
                is_static_sliding_cache = isinstance(past_key_values, StaticCache) and all(past_key_values.is_sliding)
                if not is_static_sliding_cache or sequence_length > target_length:
                    sliding_attend_mask = torch.arange(target_length, device=cache_position.device) <= (
                        cache_position.reshape(-1, 1) - text_config.sliding_window
                    )
                    diagonal_attend_mask.bitwise_or_(sliding_attend_mask)
            causal_mask *= diagonal_attend_mask
            causal_mask = causal_mask[None, None, :, :].expand(batch_size, 1, -1, -1)
            if attention_mask is not None:
                causal_mask = causal_mask.clone()  # copy to contiguous memory for in-place edit
                if attention_mask.shape[-1] > target_length:
                    attention_mask = attention_mask[:, :target_length]
                mask_length = attention_mask.shape[-1]
                padding_mask = causal_mask[:, :, :, :mask_length] + attention_mask[:, None, None, :].to(
                    causal_mask.device
                )
                padding_mask = padding_mask == 0
                causal_mask[:, :, :, :mask_length] = causal_mask[:, :, :, :mask_length].masked_fill(
                    padding_mask, min_dtype
                )
        return causal_mask

### PhiForCausalLM

PhiForCausalLMは、Phimoeを使用してテキスト生成を行うためのクラス

load_balancing_loss_funcは、MoEモデルの訓練時に使用する補助損失関数

- 均等にトークンを割り振るために使用する
- 特定のエキスパートにトークンが集中すると損失が大きくなる

$$
L_{\text{aux}} \propto \sum_{i=1}^N f_i \cdot P_i
$$

- $f_i$: エキスパート$i$に割り当てられたトークンの割合
- $P_i$: ルーターがエキスパート$i$を選んだ確率の平均値

In [ ]:
# Copied from transformers.models.qwen2_moe.modeling_qwen2_moe.load_balancing_loss_func
def load_balancing_loss_func(
    gate_logits: Union[torch.Tensor, tuple[torch.Tensor], None],
    num_experts: Optional[int] = None,
    top_k=2,
    attention_mask: Optional[torch.Tensor] = None,
) -> Union[torch.Tensor, int]:
    r"""
    Computes auxiliary load balancing loss as in Switch Transformer - implemented in Pytorch.

    See Switch Transformer (https://huggingface.co/papers/2101.03961) for more details. This function implements the loss
    function presented in equations (4) - (6) of the paper. It aims at penalizing cases where the routing between
    experts is too unbalanced.

    Args:
        gate_logits:
            Logits from the `gate`, should be a tuple of model.config.num_hidden_layers tensors of
            shape [batch_size X sequence_length, num_experts].
        num_experts:
            Number of experts
        top_k:
            The number of experts to route per-token, can be also interpreted as the `top-k` routing
            parameter.
        attention_mask (`torch.Tensor`, *optional*):
            The attention_mask used in forward function
            shape [batch_size X sequence_length] if not None.

    Returns:
        The auxiliary loss.
    """

    # 1. 入力を検証

    if gate_logits is None or not isinstance(gate_logits, tuple):
        return 0

    # 2. ゲートのロジットがタプルの場合は結合

    if isinstance(gate_logits, tuple):
        compute_device = gate_logits[0].device
        concatenated_gate_logits = torch.cat([layer_gate.to(compute_device) for layer_gate in gate_logits], dim=0)

    # 3. Softmaxでロジットを確率に変換

    routing_weights = torch.nn.functional.softmax(concatenated_gate_logits, dim=-1)

    # 4. Top-Kのエキスパートを選択

    _, selected_experts = torch.topk(routing_weights, top_k, dim=-1)

    # 5. 選択されたエキスパートのマスクを作成

    expert_mask = torch.nn.functional.one_hot(selected_experts, num_experts)

    # 6. アテンションマスクに基づいて負荷分散損失を計算

    if attention_mask is None:
        # アテンションマスクがない場合

        # 各エキスパートにルーティングされたトークンの割合を計算
        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.mean(expert_mask.float(), dim=0)

        # 各エキスパートへのルーティングの確率の平均を計算
        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.mean(routing_weights, dim=0)
    else:
        # アテンションマスクがある場合、パディングを除外して計算

        batch_size, sequence_length = attention_mask.shape
        num_hidden_layers = concatenated_gate_logits.shape[0] // (batch_size * sequence_length)

        # Compute the mask that masks all padding tokens as 0 with the same shape of expert_mask
        expert_attention_mask = (
            attention_mask[None, :, :, None, None]
            .expand((num_hidden_layers, batch_size, sequence_length, top_k, num_experts))
            .reshape(-1, top_k, num_experts)
            .to(compute_device)
        )

        # Compute the percentage of tokens routed to each experts
        tokens_per_expert = torch.sum(expert_mask.float() * expert_attention_mask, dim=0) / torch.sum(
            expert_attention_mask, dim=0
        )

        # Compute the mask that masks all padding tokens as 0 with the same shape of tokens_per_expert
        router_per_expert_attention_mask = (
            attention_mask[None, :, :, None]
            .expand((num_hidden_layers, batch_size, sequence_length, routing_weights.shape[1]))
            .reshape(-1, routing_weights.shape[1])
            .to(compute_device)
        )

        # Compute the average probability of routing to these experts
        router_prob_per_expert = torch.sum(routing_weights * router_per_expert_attention_mask, dim=0) / torch.sum(
            router_per_expert_attention_mask, dim=0
        )

    device_index = routing_weights.device.index if routing_weights.device.index is not None else 0
    rank = routing_weights.shape[1] * int(device_index)

    # 最終的な負荷分散損失を計算
    # L = sum_i (E_i * P_i) * num_experts

    overall_loss = torch.sum(
        tokens_per_expert[:, rank : rank + routing_weights.shape[1]] * router_prob_per_expert.unsqueeze(0)
    )
    return overall_loss * num_experts

In [ ]:
class PhimoeForCausalLM(PhimoePreTrainedModel, GenerationMixin):
    _tied_weights_keys = ["lm_head.weight"]

    def __init__(self, config):
        logger.info(f"PhimoeForCausalLMを初期化開始 {config.vocab_size=} {config.hidden_size=} {config.num_local_experts=} {config.num_experts_per_tok=} {config.router_aux_loss_coef=}")

        super().__init__(config)

        # Phimoeモデル本体を初期化
        self.model = PhimoeModel(config)

        self.vocab_size = config.vocab_size

        # 隠れ層を語彙サイズにマッピングする線形層
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=self.config.lm_head_bias)

        # MoEの補助損失（負荷分散損失）の重みを制御する係数
        self.router_aux_loss_coef = config.router_aux_loss_coef

        # 補助損失の計算に必要なエキスパート数とトークンごとに使用するエキスパート数
        self.num_experts = config.num_local_experts
        self.num_experts_per_tok = config.num_experts_per_tok
        # Initialize weights and apply final processing
        self.post_init()

        logger.info(f"PhimoeForCausalLMを初期化完了")

    @can_return_tuple
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[Cache] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        output_router_logits: Optional[bool] = None,
        cache_position: Optional[torch.LongTensor] = None,
        logits_to_keep: Union[int, torch.Tensor] = 0,
        **kwargs,
    ) -> MoeCausalLMOutputWithPast:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
            config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
            (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Example:
        ```python
        >>> from transformers import AutoTokenizer, PhimoeForCausalLM
        >>> model = PhimoeForCausalLM.from_pretrained("microsoft/Phi-3.5-MoE-instruct")
        >>> tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-MoE-instruct")
        >>> prompt = "Hey, are you conscious? Can you talk to me?"
        >>> inputs = tokenizer(prompt, return_tensors="pt")
        >>> # Generate
        >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
        >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        "Hey, are you conscious? Can you talk to me?\nI'm not conscious, but I can talk to you."
        ```"""

        logger.info(f"PhimoeForCausalLMの順伝播開始 {input_ids.shape if input_ids is not None else None=} {attention_mask.shape if attention_mask is not None else None=} {position_ids=} {past_key_values is not None=} {inputs_embeds.shape if inputs_embeds is not None else None=} {labels.shape if labels is not None else None=} {use_cache=} {output_attentions=} {output_hidden_states=} {output_router_logits=} {cache_position=} {logits_to_keep=}")

        if (
            use_cache
            and self.config.rope_scaling
            and cache_position is not None
            and cache_position[0] == self.config.original_max_position_embeddings
        ):
            logger.warning(
                f"If you are not using the generate method, you may encounter nonsensical outputs after the {self.config.original_max_position_embeddings}th token, as the KV cache needs to be recomputed."
            )
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_router_logits = (
            output_router_logits if output_router_logits is not None else self.config.output_router_logits
        )

        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )

        # 1. PhimoeModelの順伝播を実行

        # decoder outputs consists of (dec_features, layer_state, dec_hidden, dec_attn)
        outputs: MoeModelOutputWithPast = self.model(
            input_ids=input_ids, # (1, 130)
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            output_router_logits=output_router_logits,
            cache_position=cache_position,
        )

        hidden_states = outputs.last_hidden_state

        # 一番後ろのlogits_to_keep個のロジットのみを計算ためのインデックスを作成
        # Only compute necessary logits, and do not upcast them to float if we are not computing the loss
        slice_indices = slice(-logits_to_keep, None) if isinstance(logits_to_keep, int) else logits_to_keep

        # 2. ロジットを計算

        # (1, 130, 4096) -> (1, 1, 4096) -> (1, 1, 32064)
        logits = self.lm_head(hidden_states[:, slice_indices, :])

        # 3. 訓練時は損失を計算

        loss = None
        if labels is not None:
            loss = self.loss_function(logits, labels, self.vocab_size, **kwargs)

        aux_loss = None
        if output_router_logits:
            # MoEの補助損失を計算
            aux_loss = load_balancing_loss_func(
                outputs.router_logits,
                self.num_experts,
                self.num_experts_per_tok,
                attention_mask,
            )
            if labels is not None:
                loss += self.router_aux_loss_coef * aux_loss.to(loss.device)  # make sure to reside in the same device

        logger.info(f"PhimoeForCausalLMの順伝播完了 {logits.shape=} {loss.item() if loss is not None else None=} {aux_loss.item() if aux_loss is not None else None=}")

        return MoeCausalLMOutputWithPast(
            loss=loss,
            aux_loss=aux_loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            router_logits=outputs.router_logits,
        )

    # Copied from transformers.models.phi3.modeling_phi3.Phi3ForCausalLM.prepare_inputs_for_generation
    def prepare_inputs_for_generation(
        self,
        input_ids,
        past_key_values=None,
        attention_mask=None,
        inputs_embeds=None,
        cache_position=None,
        position_ids=None,
        use_cache=True,
        logits_to_keep=None,
        **kwargs,
    ):
        # Overwritten -- this model may need to switch between short and long rope, invalidating the cache in the
        # process

        # When the first time input length reached long and short factor switching point, enforce re-compute cache
        # It will cause downside of slower at this single token position, however, better than current failure.
        if (
            past_key_values
            and self.config.rope_scaling
            and input_ids.shape[1] >= self.config.original_max_position_embeddings + 1
        ):
            past_length = cache_position[0]
            if past_length <= self.config.original_max_position_embeddings:
                past_key_values = None

        model_inputs = super().prepare_inputs_for_generation(
            input_ids=input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            cache_position=cache_position,
            position_ids=position_ids,
            use_cache=use_cache,
            logits_to_keep=logits_to_keep,
            **kwargs,
        )
        return model_inputs

### 推論

In [ ]:
tokenizer = LlamaTokenizerFast.from_pretrained("microsoft/Phi-3.5-MoE-instruct") 
tokenizer

In [ ]:
model = PhimoeForCausalLM.from_pretrained( 
    "microsoft/Phi-3.5-MoE-instruct",  
    device_map="cuda",  
    trust_remote_code=False,  
    dtype=torch.bfloat16,
    load_in_4bit=True,
) 

In [ ]:
messages = [{"role": "user", "content": "Hello!"}] 

In [ ]:
pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

In [ ]:
generation_args = { 
    "max_new_tokens": 1, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

output = pipe(messages, **generation_args) 
logger.info(f"生成結果: {output[0]['generated_text']}")

In [ ]:
logger.setLevel(logging.ERROR)

messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."}, 
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"}, 
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."}, 
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}, 
] 
generation_args = { 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])